# 03: Test Multi-Agent System with Genie

Comprehensive testing of the agent against all functional requirements.

**Test Scenarios**:
1. Single-domain queries (FR-001, FR-003)
2. Multi-domain queries (FR-005, FR-006)
3. Context-aware follow-ups (FR-011)
4. Proactive suggestions (FR-013)
5. Error handling (FR-008)
6. Performance (FR-012)

**Model Source**: Unity Catalog (`juan_dev.genai.retail_multi_genie_agent`)

## Setup

In [0]:
%pip install --quiet --upgrade mlflow databricks-langchain langgraph langchain-core
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
import time
import pandas as pd

print("✅ Imports successful")

✅ Imports successful


## Load Agent from Unity Catalog

Load the agent from Unity Catalog using one of these methods:
- **Version number**: `models:/catalog.schema.model/1`
- **Alias**: `models:/catalog.schema.model@champion`
- **Latest version**: `models:/catalog.schema.model/latest`

In [0]:
from agent import AGENT

# Unity Catalog model (3-level namespace)
UC_MODEL_NAME = "juan_dev.genai.retail_multi_genie_agent"

# Choose loading method:
# Option 1: Load latest version
# model_uri = f"models:/{UC_MODEL_NAME}/latest"

# Option 2: Load specific version (uncomment to use)
# model_version = "1"
# model_uri = f"models:/{UC_MODEL_NAME}/{model_version}"

# Option 3: Load by alias (uncomment to use)
model_alias = "champion"  # or "staging", "production", etc.
model_uri = f"models:/{UC_MODEL_NAME}@{model_alias}"
print(f"Loading agent from Unity Catalog: {model_uri}")

AGENT = mlflow.pyfunc.load_model(model_uri)

print(f"✅ Agent loaded successfully from Unity Catalog")
print(f"   Model: {UC_MODEL_NAME}")

## Helper Functions

In [0]:
def query_agent(query: str, conversation_history: list = None) -> dict:
    """Query agent and return result"""
    import pandas as pd
    
    messages = conversation_history or []
    messages.append({"role": "user", "content": query})
    
    # Create input matching the input_example format from logging
    input_data = {
        "input": messages
    }
    
    start_time = time.time()
    
    # Try calling predict with the dict directly first
    try:
        response = AGENT.predict(input_data)
    except Exception as e:
        print(f"Direct dict call failed: {e}")
        # Fall back to DataFrame
        input_df = pd.DataFrame([input_data])
        response = AGENT.predict(input_df)
    
    elapsed_ms = (time.time() - start_time) * 1000
    
    # Extract response from PyFunc output
    # Response format from ResponsesAgent: {output: [{text: "...", id: "..."}], ...}
    if isinstance(response, pd.DataFrame):
        output = response.iloc[0]['output']
    elif isinstance(response, dict):
        output = response.get('output', response)
    else:
        output = response
    
    # Handle different output formats
    if isinstance(output, list) and len(output) > 0:
        if isinstance(output[0], dict):
            response_text = output[0].get('text', str(output[0]))
        else:
            response_text = str(output[0])
    elif isinstance(output, dict):
        response_text = output.get('text', str(output))
    else:
        response_text = str(output)
    
    # Add assistant response to conversation
    messages.append({"role": "assistant", "content": response_text})
    
    return {
        "query": query,
        "response": response_text,
        "messages": messages,
        "elapsed_ms": elapsed_ms
    }

def print_result(result: dict):
    """Pretty print result"""
    print(f"\nQuery: {result['query']}")
    print(f"Response: {result['response']}")
    print(f"Time: {result['elapsed_ms']:.0f}ms")

print("✅ Helper functions ready")

## Test 1: Single-Domain Query (FR-001, FR-003)

Test basic inventory query.

In [0]:
result = query_agent("What products are at risk for overstock?")
print_result(result)

# Validations
assert result['response'], "Should have response"
assert result['elapsed_ms'] < 90000, "Should complete within 90s (Genie timeout)"
assert "inventory" in result['response'].lower() or "overstock" in result['response'].lower(), "Should address inventory"

print("\n✅ Test 1 PASSED")

## Test 2: Multi-Domain Query (FR-005, FR-006)

Test query spanning customer behavior and inventory.

In [0]:
result = query_agent(
    "What products are frequently abandoned in carts and do we have inventory issues with those items?"
)
print_result(result)

# Validations
assert result['response'], "Should have response"
assert result['elapsed_ms'] < 90000, "Should complete within 90s"

# Check for both domains being addressed
response_lower = result['response'].lower()
has_customer_behavior = any(keyword in response_lower for keyword in ["abandon", "cart", "customer"])
has_inventory = any(keyword in response_lower for keyword in ["inventory", "stock", "overstock", "stockout"])

print(f"\nDomain coverage:")
print(f"  Customer Behavior: {'✅' if has_customer_behavior else '❌'}")
print(f"  Inventory: {'✅' if has_inventory else '❌'}")

assert has_customer_behavior or has_inventory, "Should address at least one domain"

print("\n✅ Test 2 PASSED")

## Test 3: Context-Aware Follow-Up (FR-011)

Test conversation history and context understanding.

In [0]:
# First query
result1 = query_agent("What are the top customers by purchase amount?")
print_result(result1)

# Follow-up using context
conversation_history = result1['messages']
result2 = query_agent("What products do they purchase most frequently?", conversation_history)
print_result(result2)

# Validations
assert result2['response'], "Should have follow-up response"
assert len(conversation_history) >= 4, "Should maintain conversation history"

print(f"\nConversation length: {len(conversation_history)} messages")
print("\n✅ Test 3 PASSED")

## Test 4: Error Handling (FR-008)

Test graceful handling of out-of-scope queries.

In [0]:
result = query_agent("What is the weather forecast for next week?")
print_result(result)

# Validations
assert result['response'], "Should provide response"
# Agent should politely explain it can't answer weather questions or provide guidance

print("\n✅ Test 4 PASSED")

## Test 5: Customer Behavior Domain

Test specific customer behavior queries.

In [0]:
customer_queries = [
    "What is the cart abandonment rate?",
    "Which customer segments have the highest purchase frequency?",
    "Show me top products by cart abandonment"
]

for query in customer_queries:
    result = query_agent(query)
    print(f"\nQuery: {query}")
    print(f"Time: {result['elapsed_ms']:.0f}ms")
    print(f"Response preview: {result['response'][:200]}...")
    assert result['response'], f"Should have response for: {query}"

print("\n✅ Test 5 PASSED - Customer behavior queries working")

## Test 6: Inventory Domain

Test specific inventory queries.

In [0]:
inventory_queries = [
    "What items are at risk of stockout?",
    "Show me products with high inventory turnover",
    "Which products have overstock issues?"
]

for query in inventory_queries:
    result = query_agent(query)
    print(f"\nQuery: {query}")
    print(f"Time: {result['elapsed_ms']:.0f}ms")
    print(f"Response preview: {result['response'][:200]}...")
    assert result['response'], f"Should have response for: {query}"

print("\n✅ Test 6 PASSED - Inventory queries working")

## Test 7: Performance (FR-012)

Verify complex queries complete within acceptable time (90s for Genie).

In [0]:
complex_queries = [
    "Analyze cart abandonment patterns and correlate with inventory stockouts",
    "What products have high cart abandonment and low inventory?",
    "Show customer segments most affected by inventory constraints"
]

for query in complex_queries:
    result = query_agent(query)
    print(f"\nQuery: {query[:60]}...")
    print(f"Time: {result['elapsed_ms']:.0f}ms")
    assert result['elapsed_ms'] < 90000, f"Query exceeded 90s: {result['elapsed_ms']}ms"

print("\n✅ Test 7 PASSED - All queries under 90s")

## Test Summary

Run this cell to see overall test results.

In [0]:
print("="*60)
print("TEST SUMMARY - Multi-Agent Genie System")
print("="*60)
print(f"Model: {UC_MODEL_NAME}")
print(f"Source: {model_uri}")
print("="*60)
print("✅ Test 1: Single-domain queries (FR-001, FR-003)")
print("✅ Test 2: Multi-domain queries (FR-005, FR-006)")
print("✅ Test 3: Context-aware follow-ups (FR-011)")
print("✅ Test 4: Error handling (FR-008)")
print("✅ Test 5: Customer behavior domain queries")
print("✅ Test 6: Inventory domain queries")
print("✅ Test 7: Performance under 90s (FR-012)")
print("="*60)
print("\n🎉 All tests completed!")
print("\nAgent is ready for production deployment.")

## Set Model Alias (Optional)

Set an alias like `champion`, `staging`, or `production` for easier model management.

In [0]:
from mlflow import MlflowClient

client = MlflowClient()

# Get the latest version number
latest_version = client.get_latest_versions(UC_MODEL_NAME, stages=["None"])[0].version

# Set alias (uncomment to use)
# alias_name = "champion"  # or "staging", "production", etc.
# client.set_registered_model_alias(UC_MODEL_NAME, alias_name, latest_version)
# print(f"✅ Set alias '{alias_name}' to version {latest_version}")
# print(f"   Load with: models:/{UC_MODEL_NAME}@{alias_name}")

print(f"Latest version: {latest_version}")
print(f"\nAvailable aliases:")
model_details = client.get_registered_model(UC_MODEL_NAME)
if hasattr(model_details, 'aliases') and model_details.aliases:
    for alias, version in model_details.aliases.items():
        print(f"  - {alias} → version {version}")
else:
    print("  (no aliases set yet)")

## Next Steps

1. ✅ Tests passed - agent is working correctly
2. Set model alias (e.g., `champion`) for production tracking
3. Deploy to Model Serving endpoint:
   - Use Unity Catalog model: `juan_dev.genai.retail_multi_genie_agent`
   - Enable auto-scaling and authentication passthrough
4. Create evaluation dataset for MLflow evaluation
5. Set up monitoring and alerting